In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import pickle, json, sys, os, glob
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy import optimize
import subprocess

from astropy.coordinates import SkyCoord
from lstchain.io.config  import get_standard_config
from ctapipe.io          import read_table
import tables

# Other auxiliar scripts
sys.path.insert(0, os.getcwd() + "/../scripts/")
import auxiliar as aux
import geometry as geom
import lstpipeline

import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

In [14]:
""" Source name in order to just complete the results file, and
in order to improve run organization."""
source_name = "crab"

# Standard paths for data in the IT cluster ---------
root_dl1 = "/fefs/aswg/data/real/DL1/*/v0.*/tailcut84/"
root_rfs = "/fefs/aswg/data/models/AllSky/20230927_v0.10.4_crab_tuned/"
root_mcs = "/fefs/aswg/data/mc/DL2/AllSky/20230927_v0.10.4_crab_tuned/TestingDataset/"

# Root path of this script
root = os.getcwd() + "/"
# Path to store the configuration file we are going to use
config_file = root + "config/standard_config.json"
# Path to store objects
root_objects = root + f"objects/"
# Data main directory
root_data = root + f"../../data/cherenkov_transparency_corrections/{source_name}/"

# Directories for the data
dir_dl1b_scaled = root_data + "dl1_scaled/"
dir_dl1m_scaled = root_data + "dl1_merged_scaled/"
dir_dl2_scaled  = root_data + "dl2_scaled/"
dir_dl2         = root_data + "dl2/"
dir_dl3_scaled_base = root_data + "__dl3_scaled_total/"
dir_dl3_base    = root_data + "__dl3_total/"
dir_irfs        = root_data + "irfs/"

In [22]:
RRR = [2853, 2922, 2923, 2924, 2934, 2950, 2959, 2960, 2965, 2967, 2968, 2969, 2971, 2972, 2974, 2975, 2977, 2988, 2989, 2990, 2991, 2992, 3009, 3011, 3012, 3087, 3088, 3089, 3093, 3095, 3096, 3097, 3098, 3100, 3144, 3169, 3170, 3231, 3232, 3235, 3236, 3237, 3243, 3244, 3245, 3265, 3266, 3267, 3268, 3269, 3270, 3271, 3272, 3273, 3274, 3275, 3276, 3277, 3278, 3318, 3319, 3320, 3321, 3328, 3329, 3338, 3339, 3340, 3355, 3356, 3382, 3515, 3516, 3583, 3584, 3585, 3586, 3598, 3599, 3600, 3601, 3631, 3632, 3633, 3634, 3635, 3636, 3637, 3638, 3639, 3650, 3651, 3652, 3653, 3655, 3672, 3673, 3674, 3675, 3676, 3677, 3678, 3679, 3680, 3683, 3684, 3703, 3704, 3705, 3706, 3707, 3708, 3709, 3710, 3711, 3712, 3725, 3730, 3894, 3895, 3953, 3956, 3957, 3959, 3961, 3962, 3983, 4008, 4009, 4010, 4011, 4012, 4013, 4014, 4015, 4033, 4034, 4035, 4036, 4067, 4068, 4069, 4073, 4074, 4075, 4086, 4087, 4088, 4089, 4090, 4094, 4125, 4126, 4127, 4128, 4129, 4145, 4146, 4147, 4148, 4149, 7084, 7087, 7088, 7097, 7099, 7100, 7101, 7102, 7106, 7133, 7134, 7135, 7136, 7137, 7140, 7141, 7142, 7143, 7168, 7169, 7170, 7172, 7173, 7174, 7196, 7197, 7201, 7202, 7227, 7228, 7230, 7231, 7232, 7233, 7234, 7235, 7236, 7253, 7254, 7275, 7276, 7277, 7278, 7279, 7282, 7301, 7302, 7304]

dl3_runs_individual_scaled = glob.glob(dir_dl3_scaled_base + "*/*.fits")
dl3_runs_individual = glob.glob(dir_dl3_base + "*/*.fits")

dl3_runs_individual_scaled

rrr_n = [int(f.split("Run")[-1].split(".")[0]) for f in dl3_runs_individual]
rrr_s = [int(f.split("Run")[-1].split(".")[0]) for f in dl3_runs_individual_scaled]

common_rrr = []
for rrr in rrr_n:
    if rrr in rrr_s:
        common_rrr.append(rrr)

In [29]:
dir_dl3_scaled_ = root_data + "dl3_scaled_0/"
dir_dl3_    = root_data + "dl3_0/"

for dl3_runs, dir_move, rrr in zip([dl3_runs_individual_scaled, dl3_runs_individual], [dir_dl3_scaled_, dir_dl3_], [rrr_s, rrr_n]):
    
    
    command = f"cp"
   
    for i in range(len(dl3_runs)):

        if rrr[i] in RRR and rrr[i] in common_rrr:
        
            command = command + f" {dl3_runs[i]}"
    
    # moving to main folder
    command = command + " " + dir_move
    
    subprocess.run(command, shell=True)


In [10]:
# dl3_runs_individual_scaled = glob.glob(dir_dl3_scaled_base + "*/*.fits")
# dl3_runs_individual = glob.glob(dir_dl3_base + "*/*.fits")

# for dl3_runs, dir_move in zip([dl3_runs_individual_scaled, dl3_runs_individual], [dir_dl3_scaled_base, dir_dl3_base]):
#     command = f"cp"
    
#     for file in dl3_runs:
#         command = command + f" {file}"
    
#     # moving to main folder
#     command = command + " " + dir_move
    
#     subprocess.run(command, shell=True)

### Add DL3 index files

In [30]:
command = f"lstchain_create_dl3_index_files --input-dl3-dir {dir_dl3_} --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

command = f"lstchain_create_dl3_index_files --input-dl3-dir {dir_dl3_scaled_} --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

logger.info(f"\nFinished with the dl3 process")

lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3_0/ --overwrite
2024-05-19 13:01:58,463 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2934 does not contain HDU BACKGROUND
2024-05-19 13:01:58,463 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2934 does not contain HDU PSF
2024-05-19 13:01:58,474 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2950 does not contain HDU BACKGROUND
2024-05-19 13:01:58,475 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2950 does not contain HDU PSF
2024-05-19 13:01:58,485 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2967 does not contain HDU BACKGROUND
2024-05-19 13:01:58,485 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2967 does not con

In [11]:
command = f"lstchain_create_dl3_index_files --input-dl3-dir {dir_dl3_base} --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

command = f"lstchain_create_dl3_index_files --input-dl3-dir {dir_dl3_scaled_base} --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

logger.info(f"\nFinished with the dl3 process")

lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3_4/ --overwrite
2024-05-16 07:49:18,436 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 3095 does not contain HDU BACKGROUND
2024-05-16 07:49:18,437 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 3095 does not contain HDU PSF
2024-05-16 07:49:18,518 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 3096 does not contain HDU BACKGROUND
2024-05-16 07:49:18,518 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 3096 does not contain HDU PSF
2024-05-16 07:49:18,621 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 3232 does not contain HDU BACKGROUND
2024-05-16 07:49:18,622 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 3232 does not con

In [10]:
direc = "/fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_paper/"

command = f"lstchain_create_dl3_index_files --input-dl3-dir {direc} --file-pattern dl3*fits --overwrite"
logger.info(command)
subprocess.run(command, shell=True)

lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_paper/ --file-pattern dl3*fits --overwrite
2024-03-13 14:49:31,396 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2914 does not contain HDU BACKGROUND
2024-03-13 14:49:31,396 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2914 does not contain HDU PSF
2024-03-13 14:49:31,538 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2929 does not contain HDU BACKGROUND
2024-03-13 14:49:31,539 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2929 does not contain HDU PSF
2024-03-13 14:49:31,548 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2930 does not contain HDU BACKGROUND
2024-03-13 14:49:31,548 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2930 does not contain HDU PSF
2024-03-13 14:49:32,304 ERROR [lstchain.high_level.hdu

CompletedProcess(args='lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_paper/ --file-pattern dl3*fits --overwrite', returncode=0)

In [6]:
# direc = "/fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3/"

# command = f"lstchain_create_dl3_index_files --input-dl3-dir {direc} --overwrite"
# logger.info(command)
# subprocess.run(command, shell=True)

lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3/ --overwrite
2024-03-11 16:21:59,923 CRITICAL [lstchain.FITSIndexWriter] (application.inner): Bad config encountered during initialization: Path "/fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3" does not exist


CompletedProcess(args='lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3/ --overwrite', returncode=1)

In [7]:
# direc = "/fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_scaled/"

# command = f"lstchain_create_dl3_index_files --input-dl3-dir {direc} --overwrite"
# logger.info(command)
# subprocess.run(command, shell=True)

lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_scaled/ --overwrite
2024-03-11 16:22:11,155 WARNING [lstchain.FITSIndexWriter] (lstchain_create_dl3_index_files.setup): Overwriting /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_scaled/hdu-index.fits.gz
2024-03-11 16:22:11,156 WARNING [lstchain.FITSIndexWriter] (lstchain_create_dl3_index_files.setup): Overwriting /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_scaled/obs-index.fits.gz
2024-03-11 16:22:11,572 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2914 does not contain HDU BACKGROUND
2024-03-11 16:22:11,572 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2914 does not contain HDU PSF
2024-03-11 16:22:11,679 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 2929 does not contain HDU BACKGROUND
2024-03-11 16:22:11,679 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hd

CompletedProcess(args='lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_scaled/ --overwrite', returncode=0)